In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset

Dataset URL: https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset
License(s): other


In [4]:
!unzip usa-real-estate-dataset.zip

Archive:  usa-real-estate-dataset.zip
  inflating: realtor-data.zip.csv    


In [5]:
!unzip model_checkpoints.zip

Archive:  model_checkpoints.zip
   creating: results/
   creating: results/checkpoint-2000/
  inflating: results/checkpoint-2000/README.md  
  inflating: results/checkpoint-2000/rng_state.pth  
  inflating: results/checkpoint-2000/tokenizer.json  
  inflating: results/checkpoint-2000/special_tokens_map.json  
  inflating: results/checkpoint-2000/adapter_config.json  
  inflating: results/checkpoint-2000/training_args.bin  
  inflating: results/checkpoint-2000/adapter_model.safetensors  
  inflating: results/checkpoint-2000/tokenizer_config.json  
  inflating: results/checkpoint-2000/optimizer.pt  
  inflating: results/checkpoint-2000/scaler.pt  
  inflating: results/checkpoint-2000/scheduler.pt  
  inflating: results/checkpoint-2000/trainer_state.json  
   creating: results/checkpoint-1000/
  inflating: results/checkpoint-1000/README.md  
  inflating: results/checkpoint-1000/rng_state.pth  
  inflating: results/checkpoint-1000/tokenizer.json  
  inflating: results/checkpoint-1000/speci

In [2]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

df = pd.read_csv("realtor-data.csv")
df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


### Data preprocessing

In [ ]:
df.shape,df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB


((2226382, 12), None)

In [3]:
df["prev_sold_date"] = pd.to_datetime(df["prev_sold_date"], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   brokered_by     float64       
 1   status          object        
 2   price           float64       
 3   bed             float64       
 4   bath            float64       
 5   acre_lot        float64       
 6   street          float64       
 7   city            object        
 8   state           object        
 9   zip_code        float64       
 10  house_size      float64       
 11  prev_sold_date  datetime64[ns]
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 203.8+ MB


In [ ]:
df.isnull().sum()

,0
brokered_by,4533
status,0
price,1541
bed,481317
bath,511771
acre_lot,325589
street,10866
city,1407
state,8
zip_code,299


In [4]:
df.dropna(subset = ['state','price'], inplace = True)

In [ ]:
df["status"].value_counts()

,count
status,
for_sale,1388357
sold,811764
ready_to_build,24712


In [5]:
print("for sale: \n", df[df["status"] == "for_sale"].isnull().sum())
print("\n sold: \n" ,df[df["status"] == "sold"].isnull().sum())
print("\n ready to build: \n" ,df[df["status"] == "ready_to_build"].isnull().sum())

for sale: 
 brokered_by         1485
status                 0
price                  0
bed               410945
bath              408405
acre_lot          192809
street              7698
city                1306
state                  0
zip_code             284
house_size        464234
prev_sold_date    708536
dtype: int64

 sold: 
 brokered_by         3020
status                 0
price                  0
bed                69912
bath               77867
acre_lot          107613
street              2277
city                  98
state                  0
zip_code              12
house_size        103627
prev_sold_date         1
dtype: int64

 ready to build: 
 brokered_by          28
status                0
price                 0
bed                   2
bath              24712
acre_lot          24712
street              889
city                  0
state                 0
zip_code              0
house_size           11
prev_sold_date    24712
dtype: int64


#### Best context-aware strategy to handle missing values :
| Status           | Use Case             | Handling Nulls                         |
| ---------------- | -------------------- | -------------------------------------- |
| sold           | **Train model**      | Must drop rows missing critical values |
| for_sale       | **Future inference** | Fill missing values based on aggregations by state           |
| ready_to_build | **Future inference** | Tolerate missing `bed`, `bath`, etc.   |

In [6]:
# drop rows that are sold & incomplete
sold = df["status"] == "sold"

required_fields = ["price", "bed", "bath", "house_size", "prev_sold_date"]
complete_rows = df[required_fields].notna().all(axis=1)
final_mask = sold & complete_rows

df = df[~(sold & ~complete_rows)].reset_index(drop=True)

### Cleaning entries that are recorded as sold

In [7]:
df_processed = df.copy()

mask = df_processed["status"] == "sold"
sold = df_processed[mask].copy()

# fill missing values with state-level mode of brokers
sold["brokered_by"] = sold.groupby("state")["brokered_by"].transform(
    lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else "Unknown"
)

# fill missing values with state-level mode of acre_lot
sold["acre_lot"] = sold.groupby("state")["acre_lot"].transform(lambda x: x.fillna(x.mean()))

df_processed.loc[mask, ["brokered_by", "acre_lot"]] = sold[["brokered_by", "acre_lot"]]

In [8]:
df_processed.loc[mask, "street"] = df_processed.loc[mask, "street"].fillna("Unknown Street")
df_processed.loc[mask, "city"] = df_processed.loc[mask, "city"].fillna("Unknown City")
df_processed.loc[mask, "zip_code"] = df_processed.loc[mask, "zip_code"].fillna("00000").astype(str).str.zfill(5)

### Cleaning entries that are recorded as for sale

In [9]:
forSale_mask = df_processed["status"] == "for_sale"
for_sale = df_processed[forSale_mask].copy()

# fill  missing values with state-level mode of ["bed", "bath", "brokered_by"]
for col in ["bed", "bath", "brokered_by"]:
    for_sale[col] = for_sale.groupby("state")[col].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna("Unknown" if col == "brokered_by" else 0)
    )

# fill missing values with state-level mode of ["acre_lot", "house_size"]
for col in ["acre_lot", "house_size"]:
    for_sale[col] = for_sale.groupby("state")[col].transform(lambda x: x.fillna(x.mean()))

df_processed.loc[forSale_mask, for_sale.columns] = for_sale

In [10]:
df_processed.loc[forSale_mask, "street"] = df_processed.loc[forSale_mask, "street"].fillna("Unknown Street")
df_processed.loc[forSale_mask, "city"] = df_processed.loc[forSale_mask, "city"].fillna("Unknown City")
df_processed.loc[forSale_mask, "zip_code"] = df_processed.loc[forSale_mask, "zip_code"].fillna("00000").astype(str).str.zfill(5)
df_processed = df_processed[~(forSale_mask & df_processed["house_size"].isna())].reset_index(drop=True)

### Cleaning entries that are recorded as ready to build

In [11]:
rtb_mask = df_processed["status"] == "ready_to_build"
rtb = df_processed[rtb_mask].copy()

# fill  missing values with state-level mode of ["bed", "bath", "brokered_by"]
for col in ["bed", "bath", "brokered_by"]:
    rtb[col] = rtb.groupby("state")[col].transform(
        lambda x: x.fillna(x.mode().iloc[0]) if not x.mode().empty else x.fillna("Unknown" if col == "brokered_by" else 0)
    )

# fill missing values with state-level mode of ["acre_lot", "house_size"]
for col in ["acre_lot", "house_size"]:
    rtb[col] = rtb.groupby("state")[col].transform(lambda x: x.fillna(x.mean()))

df_processed.loc[rtb_mask, rtb.columns] = rtb

In [12]:
df_processed.loc[rtb_mask, "street"] = df_processed.loc[rtb_mask, "street"].fillna("Unknown Street")
df_processed.loc[rtb_mask, "acre_lot"] = df_processed.loc[rtb_mask, "acre_lot"].fillna(0)

In [13]:
df_processed.isnull().sum()

,0
brokered_by,0
status,0
price,0
bed,0
bath,0
acre_lot,0
street,0
city,0
state,0
zip_code,0


### Correcting datatypes

In [14]:
columns_to_convert = ["bed", "bath", "brokered_by"]
df_processed[columns_to_convert] = df_processed[columns_to_convert].astype(int)

#### Nice! now the data is clean and ready to be worked on
prev_sold_date will remain with missing values because of couple of reasons:
- The prev_sold_date is only applicable to properties that have actually been sold. For statuses like for_sale or ready_to_build, this field is naturally and logically missing (the house hasn’t been sold before), so there’s no valid date to provide. Filling it with a fake value like "Unknown" or a placeholder date would introduce noise and mislead the model into treating it as a meaningful historical date.
- I'm planning to use prev_sold_date only in prompt construction for fine-tuning and inference, rather than as a tabular feature.
- To prevent Bias and Data Leakage

#### missing values handling approaches summary
| Column                               | Missing? | Notes                                         |
| ------------------------------------ | -------- | --------------------------------------------- |
| brokered_by                        | ❌        | Filled dropped based on `status` logic    |
| status                            | ❌        | Always present                                |
| price                           | ❌        | Dropped all rows with missing values          |
| bed / bath                      | ❌        | filled or dropped based on `status` logic    |
| acre_lot                           | ❌        | Filled with mean or `0` (neutral default)             |
| street/ city/ zip_code | ❌        | Standard placeholder fills                    |
|state  | ❌ | dropped all rows with missing values
| house_size                        | ❌        | filled or dropped based on `status` logic    |
| prev_sold_date                     | ✅        | Left as-is to be handled in prompt formatting |


### Convert sold Data to Prompt-Response Pairs

In [ ]:
import json

sold_data = df_processed[df_processed['status'] == 'sold'].copy()

# Format each row into instruction-style prompt
records = []
for _, row in sold_data.iterrows():
    prompt = f"""### Instruction:
Estimate the price of this house given the listing details.

### Input:
- Beds: {row['bed']}
- Baths: {row['bath']}
- Lot Size (acres): {row['acre_lot']}
- House Size (sqft): {row['house_size']}
- Broker: {row['brokered_by']}
- City: {row['city']}
- State: {row['state']}
- Zip Code: {row['zip_code']}
- Status: {row['status']}
- Previously Sold Date: {row['prev_sold_date']}

### Response:
${int(row['price'])}"""

    records.append({"text": prompt})

with open("sold_prompts.jsonl", "w") as f:
    for record in records:
        json.dump(record, f)
        f.write("\n")


### LLaMA 3.2 1B Fine-Tuning with LoRA + QLoRA (Unsloth)

In [20]:
!pip install unsloth[torch] transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 116.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23

In [ ]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig
from warnings import filterwarnings
filterwarnings('ignore')

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = 512,
    dtype = None,  # auto selects bf16 if available, else float16
    load_in_4bit = True
)

model.gradient_checkpointing_enable()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    target_modules = ["q_proj", "v_proj"],
    use_gradient_checkpointing = True,
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.7 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset,DatasetDict

sold_data = load_dataset("json", data_files="sold_prompts.jsonl", split="train")

#training on only a small random sample of the data due to limited resources
sampled_data = sold_data.shuffle(seed=42).select(range(10000))
split_data = sampled_data.train_test_split(test_size=0.1, seed=42) # Split into 90% train / 10% eval

train_dataset = split_data["train"]
eval_dataset = split_data["test"]
# small sample to test on
eval_dataset = eval_dataset.select(range(20))

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    preds = predictions.argmax(-1)

    preds = preds.flatten()
    labels = labels.flatten()

    mask = labels != -100
    preds = preds[mask]
    labels = labels[mask]

    return {
        "accuracy": accuracy_score(labels, preds)
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=1e-4,
    weight_decay=0.01,
    save_strategy="steps",
    save_steps=1000,
    logging_steps=50,
    max_steps=2000,
    fp16=True,
    push_to_hub=False,
    report_to="none",
    dataloader_num_workers=2,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_args,
    compute_metrics=compute_metrics
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,000 | Num Epochs = 2 | Total steps = 2,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 851,968/1,000,000,000 (0.09% trained)


Step,Training Loss
50,1.120800
100,0.561000
150,0.548700
200,0.543700
250,0.550900
300,0.539400
350,0.532400
400,0.531200
450,0.523900
500,0.525200


TrainOutput(global_step=2000, training_loss=0.5187151174545288, metrics={'train_runtime': 1663.3469, 'train_samples_per_second': 9.619, 'train_steps_per_second': 1.202, 'total_flos': 1.0797353115648e+16, 'train_loss': 0.5187151174545288})

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, BitsAndBytesConfig

# Path to your saved checkpoint
checkpoint_path = "/content/results/checkpoint-2000"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=checkpoint_path,     # this is your saved model directory
    max_seq_length=2048,
    dtype=None,
    quantization_config=bnb_config,
)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    args=training_args,
    compute_metrics=compute_metrics,
    max_seq_length=512 ,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
import torch

torch.cuda.empty_cache()

results = trainer.evaluate()
print("Validation Results:", results)

Validation Results: {'eval_loss': 0.49732881784439087, 'eval_model_preparation_time': 0.0021, 'eval_accuracy': 0.0, 'eval_runtime': 3.149, 'eval_samples_per_second': 6.351, 'eval_steps_per_second': 6.351}


### Use Case for for_sale and ready_to_build Data:

1. Price Prediction (Real-World Usage):

    - I'll feed these listings (for_sale/ready_build) through the trained model to predict expected sale prices.

    - This allows:

  
      Buyers to know if a house is overpriced.

  
      Sellers/agents to get price suggestions.

  
      Platforms to highlight “undervalued” listings.

      - Input: for_sale listing info (bedrooms, size, location, etc.)

      - Model output: predicted price

      - Usage: Price guidance for real-world listings

2. Model Evaluation:

    - Compare model’s predictions on recent for_sale listings with their actual sale prices once sold (delayed label).

    - This helps evaluate how well the model works in practice.

#### Note: Training Data Expansion (Later) --> After testing the model, we can gradually add for_sale data to the training data and carefully feed the model with clean and formatted trady_to_build data



## Deployment